<a href="https://colab.research.google.com/github/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/Tutorials/Tutorial%20for%20SMT%20based%20Verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial for SMT based Verification

As an example we try to verifiy the robustness of a classification Network trained on the iris dataset. We learn how to train a small neural network in pytorch and use the wheigts and biases to create a number of terms and formulas which can be solved by a SMT solver.


In [3]:
!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/DDiekmann/Applied-Verification-Lab-Neural-Networks/main/lib/iris_trainer.py'}
import iris_trainer

from torch import nn
from torch.autograd import Variable

# numpy
import numpy as np

# for plots
import matplotlib.pyplot as plt

# for measuring runtime
import time

# SMT solver
%pip install z3-solver
from z3 import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Training of Neural Network

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, number_of_neurons):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_dim, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, output_dim),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
X_train, X_test, y_train, y_test = iris_trainer.load_dataset()

iris_trainer.train_model(NeuralNetwork(input_dim=X_train.shape[1], output_dim=3, number_of_neurons=20), epochs=100, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

Shape of X (data): (150, 4)
Shape of y (target): (150,) int64
Example of x and y pair: [5.1 3.5 1.4 0.2] 0
Shape of training set X (120, 4)
Shape of test set X (30, 4)


100%|██████████| 100/100 [00:00<00:00, 785.65it/s]


Done. Accuracy: 0.8333333333333334


## Creating Formulas for the SMT solver

## Verifying robustness